### 1. Implement incremental MERGE

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "sales_delta_oct")

In [0]:
df2 = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/workspace/ecommerce/ecommerce_data_sample/sample_df.csv")



In [0]:
df2.orderBy('event_time', ascending=True) \
    .show(5)

In [0]:
deltaTable.alias('t').merge(
            df2.alias('s'),
            "t.user_session = s.user_session AND t.event_time = s.event_time") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
.execute()

In [0]:
deltaTable.toDF() \
    .orderBy("event_time", ascending=True) \
    .show(5)


### 2. Query historical versions

In [0]:
spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .table("sales_delta_oct") \
   .show(5)

In [0]:
v0 = spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load("/Volumes/workspace/ecommerce/delta/events1") \

yesterday = spark.read.format("delta") \
  .option("timestampAsOf", "2026-01-12 11:01:50") \
  .load("/Volumes/workspace/ecommerce/delta/events1")


In [0]:
yesterday.orderBy('event_time', ascending=False).show(5)

### 3. Optimize tables

In [0]:
spark.sql("OPTIMIZE sales_delta_oct ZORDER BY (event_type, user_id)")

### 4. Clean old files

In [0]:
spark.sql("VACUUM sales_delta_oct RETAIN 168 HOURS")